In [1]:
import transformers
import torch

In [2]:
gpt_tokenizer = transformers.GPT2Tokenizer.from_pretrained('gpt2')
gpt_model = transformers.GPT2LMHeadModel.from_pretrained('gpt2')

In [11]:
p_enc=gpt_model.transformer.wpe.weight

p_enc[0]/p_enc[100]

tensor([ 1.1169e+02,  1.2533e+01, -3.8949e-01, -6.0882e+00,  8.4342e+01,
        -1.7971e+02, -1.2914e+00,  3.1876e+01, -1.8420e+01,  2.7635e+01,
        -3.3555e-01,  3.2266e+01, -8.9705e-01, -1.3624e+01, -3.7434e+00,
         2.2067e+01, -6.4674e-01, -2.0057e+02,  2.1522e+01,  3.9322e-01,
        -6.5195e+00, -2.1760e+01,  5.1907e+00, -5.1310e+00,  2.1649e+00,
         1.6549e+02, -4.6503e+00, -3.5925e+01,  2.2309e+00, -7.4704e+00,
         1.0835e+01,  2.4418e+00, -3.0657e+00,  3.0457e+00, -4.3800e+00,
         6.6186e+00, -3.6847e+00, -7.7172e+00, -6.4616e+00, -3.0092e+01,
         4.5695e-01,  6.2709e+00, -8.8420e+00, -3.5151e-01, -4.6982e+01,
         3.4395e+00, -4.0568e+01, -1.8289e+01, -4.7893e+00,  6.4081e-01,
         1.0595e+01,  4.6663e+00, -3.4248e+01, -7.2450e+01,  2.7663e+00,
         2.7283e+01, -6.8191e+00,  1.9774e+01, -1.5493e+01,  4.5872e+00,
         1.3842e+01, -7.6900e+00,  8.8553e+00,  6.3136e-01, -1.3467e+00,
        -1.9485e+00,  1.8581e+00, -3.2722e+01, -1.1

In [12]:
gpt_model??

Signature:      gpt_model(*input, **kwargs)
Type:           GPT2LMHeadModel
String form:   
GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Con

In [1]:
asd=1